# Evaluate AI agents (Azure AI Agent Service) in Azure AI Foundry

## Objective


This sample demonstrates how to evaluate an AI agent (Azure AI Agent Service) on these important aspects of your agentic workflow:

- Intent Resolution: Measures how well the agent identifies the user’s request, including how well it scopes the user’s intent, asks clarifying questions, and reminds end users of its scope of capabilities.
- Tool Call Accuracy: Evaluates the agent's ability to select the appropriate tools, and process correct parameters from previous steps.
- Task Adherence: Measures how well the agent’s response adheres to its assigned tasks, according to its system message and prior steps.

For AI agents outside of Azure AI Agent Service, you can still provide th agent data in the two formats (either simple data or agent messages) specified in the individual evaluator samples:
- [Intent resolution](https://aka.ms/intentresolution-sample)
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample)
- [Task adherence](https://aka.ms/taskadherence-sample)
- [Response Completeness](https://aka.ms/rescompleteness-sample)



## Time 

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
Creating an agent using Azure AI agent service requires an Azure AI Foundry project and a deployed, supported model. See more details in [Create a new agent](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal).

For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

Important: Make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

Also, ensure you have a blob storage account with configured RBAC access for the AI Foundry Projects identity: https://github.com/MicrosoftDocs/azure-ai-docs/blob/main/articles/ai-foundry/how-to/evaluations-storage-account.md

### Prerequisite

Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for AI-assisted evaluators, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name
9) **AGENT_MODEL_DEPLOYMENT_NAME** - The deployment name of the model for your Azure AI agent, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.

### Initializing Project Client

In [83]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import FunctionTool, ToolSet

# Import your custom functions to be used as Tools for the Agent
from user_functions import user_functions

project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=os.environ["AZURE_AI_FOUNDRY_ENDPOINT"],
)

AGENT_NAME = "Seattle Tourist Assistant"

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(tools=toolset)

# Create an AI agent (Azure AI Agent Service)


In [ ]:
# Concise instructions for the Seattle Tourist Assistant
INSTRUCTIONS_PROMPT = """
You are the Seattle Tourist Assistant — a friendly, concise travel helper for visitors to Seattle, WA.
- Use available tools for facts (fetch_weather, opening_hours, send_email, convert_temperature, calculate_sum).
- For factual requests, call the matching tool and prefer tool output over guessing.
- When sending email: draft subject & body, ask for recipient confirmation, then call send_email.
- Start replies with a one-line summary, use bullets for options, and finish with a next-step suggestion.
- Ask one clarifying question if info is missing.
""".strip()

agent = project_client.agents.create_agent(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions=INSTRUCTIONS_PROMPT,
    toolset=toolset,
)


print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_QsNuI4PNKte1q9AQ1LFYxJVQ
Created agent, ID: asst_QsNuI4PNKte1q9AQ1LFYxJVQ


# Create a Thread

In [116]:
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_1mcZrsaUEFFe23uHhuwfh5ie


## Conversation with Agent
Use below cells to have conversation with the agent
- `Create Message[1]`
- `Execute[2]`

### Create Message[1]

In [117]:
# Create message to thread

MESSAGE = "Could you please email me with the details on weather in Seattle next week? and a fun number puzzle that includes addition."

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_G7hmWydeuhNVGwBBWl8ae8u1


### Execute[2]

In [118]:
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Run finished with status: RunStatus.COMPLETED
Run ID: run_8Jd5QgHYrVfjd4rNRvdBFox3


### List Messages

In [119]:
for message in project_client.agents.messages.list(thread_id=thread.id, order='asc'):
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.USER
Content: Could you please email me with the details on weather in Seattle next week? and a fun number puzzle that includes addition.
----------------------------------------
Role: MessageRole.AGENT
Content: I’ll help you get all set: I’ll prepare an email with Seattle weather for next week and a fun addition puzzle!

Could you please provide your email address so I can send the details?
----------------------------------------


In [120]:
import time as _time

# Interactive agent chat loop (place at cell index 16)
# Allows repeated user turns; press Ctrl+C to exit the loop gracefully.

def _extract_text_from_msg(msg):
    """Extract concatenated text segments from a thread message content."""
    parts = []
    for item in getattr(msg, "content", []) or msg.get("content", []):
        # Support both object attrs and dict-like items
        t = None
        if isinstance(item, dict):
            if item.get("type") == "text" and item.get("text"):
                t = item["text"].get("value")
        else:
            # object-like content
            if getattr(item, "type", None) == "text" and getattr(item, "text", None):
                text_obj = getattr(item, "text")
                t = text_obj.get("value") if isinstance(text_obj, dict) else getattr(text_obj, "value", None)
        if t:
            parts.append(t)
    return "\n".join(parts).strip()

# Seed printed message ids so we only display new messages
printed_ids = set()
for m in project_client.agents.messages.list(thread_id=thread.id, order="asc"):
    printed_ids.add(getattr(m, "id", m.get("id")))

print(f"Interactive chat with agent '{AGENT_NAME}'. Press Ctrl+C to exit.\n")

try:
    while True:
        try:
            user_input = input("You: ").strip()
        except KeyboardInterrupt:
            # Let outer handler manage exit on Ctrl+C during input
            raise

        if not user_input:
            continue

        # Create user message in the thread
        try:
            created = project_client.agents.messages.create(
                thread_id=thread.id,
                role="user",
                content=user_input,
            )
            created_id = getattr(created, "id", created.get("id"))
            print(f"[sent message id={created_id}]")
        except Exception as e:
            print(f"Failed to create message: {e}")
            continue

        # Ask the agent to process the thread
        try:
            run = project_client.agents.runs.create_and_process(
                thread_id=thread.id,
                agent_id=agent.id,
            )
        except Exception as e:
            print(f"Failed to start agent run: {e}")
            continue

        # Poll for run completion (safe short-poll loop)
        poll_interval = 1.0
        max_wait_seconds = 60
        waited = 0.0
        while getattr(run, "status", "").lower() not in ("completed", "failed") and waited < max_wait_seconds:
            _time.sleep(poll_interval)
            waited += poll_interval
            try:
                run = project_client.agents.runs.get(run.id)
            except Exception:
                break

        # Fetch and print only new messages
        for msg in project_client.agents.messages.list(thread_id=thread.id, order="asc"):
            msg_id = getattr(msg, "id", msg.get("id"))
            if msg_id in printed_ids:
                continue
            printed_ids.add(msg_id)

            role = getattr(msg, "role", msg.get("role", "unknown"))
            text = _extract_text_from_msg(msg)
            if not text:
                # Fallback printing raw content representation
                print(f"\nRole: {role}\nContent: (no text content) {getattr(msg, 'content', msg.get('content'))}\n{'-'*40}")
            else:
                print(f"\nRole: {role}\n{text}\n{'-'*40}")

        # If run failed, inform the user
        if getattr(run, "status", "").lower() == "failed":
            print(f"Agent run failed (run id={getattr(run, 'id', run.get('id'))}). Check logs or run.last_error if available.")
except KeyboardInterrupt:
    print("\nInteractive session ended by user (Ctrl+C).")

Interactive chat with agent 'Seattle Tourist Assistant'. Press Ctrl+C to exit.

[sent message id=msg_EJ7iwpOV8DzeUHAElAKbTrHh]

Role: MessageRole.USER
yes, its myeamil@email.com
----------------------------------------

Role: MessageRole.AGENT
Here's how your email will look:
- Subject: Seattle Weather Next Week & Fun Number Puzzle
- Body:
  - Weather in Seattle next week: Expect rainy conditions with an average temperature around 14°C (57°F).
  - Fun puzzle: What is 23 + 47? (Answer: 70)

Is this okay to send to myeamil@email.com, or would you like to add anything?
----------------------------------------
[sent message id=msg_Tzq1gbprjMC5kmmReZxbolQv]
Sending email to myeamil@email.com...
Subject: Seattle Weather Next Week & Fun Number Puzzle
Body:
Weather in Seattle next week: Expect rainy conditions with an average temperature around 14°C (57°F).

Fun puzzle: What is 23 + 47? (Answer: 70)

Role: MessageRole.USER
thats good, please go ahead and send.
---------------------------------

# Evaluate

### Get data from agent

In [90]:
from azure.ai.evaluation import AIAgentConverter

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

# Use the thread id associated with the run to ensure the run and thread match.
# Fallback to the previously created thread if the run does not have an associated thread id.
thread_id = run.thread_id if getattr(run, "thread_id", None) else thread.thread_id
run_id = run.id
file_name = "evaluation_data.jsonl"

# Get a single agent run data
evaluation_data_single_run = converter.convert(thread_id=thread_id, run_id=run_id)

# Run this to save thread data to a JSONL file for evaluation
# Save the agent thread data to a JSONL file
import json
evaluation_data = converter.prepare_evaluation_data(thread_ids=thread_id, filename=file_name)
print(json.dumps(evaluation_data, indent=4))

[
    {
        "query": [
            {
                "role": "system",
                "content": "You are a helpful assistant"
            },
            {
                "createdAt": "2025-08-27T04:17:28Z",
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Could you please email me with the details on weather in Seattle next week? and a fun number puzzle that includes addition."
                    }
                ]
            }
        ],
        "response": [
            {
                "createdAt": "2025-08-27T04:17:36Z",
                "run_id": "run_OjcqLPwT9U9GiNQsL7F2gxlM",
                "role": "assistant",
                "content": [
                    {
                        "type": "tool_call",
                        "tool_call_id": "call_Tf9z18WWGoLtj3iyBHPqYnxo",
                        "name": "fetch_weather",
                        "arguments": {

In [91]:
# Clean and validate evaluation data for agent evaluators
import json

def filter_valid_agent_data(filename="evaluation_data.jsonl"):
    """
    Filter evaluation data to only include entries suitable for agent evaluation.
    Removes entries that only have system messages or have malformed conversation flows.
    """
    valid_entries = []
    
    with open(filename, 'r') as f:
        for line_num, line in enumerate(f, 1):
            try:
                data = json.loads(line.strip())
                
                # Check if query has meaningful user interaction (not just system message)
                query = data.get('query', [])
                response = data.get('response', [])
                
                # Skip entries with only system messages or empty queries
                user_messages = [msg for msg in query if msg.get('role') == 'user']
                if not user_messages:
                    print(f"Skipping line {line_num}: No user messages found")
                    continue
                
                # Skip entries with empty responses
                if not response:
                    print(f"Skipping line {line_num}: Empty response")
                    continue
                
                # Check for proper agent response structure
                agent_responses = [msg for msg in response if msg.get('role') == 'assistant']
                if not agent_responses:
                    print(f"Skipping line {line_num}: No assistant responses found")
                    continue
                
                valid_entries.append(data)
                print(f"Line {line_num}: Valid agent interaction found")
                
            except json.JSONDecodeError:
                print(f"Skipping line {line_num}: Invalid JSON")
                continue
    
    print(f"\nTotal valid entries for agent evaluation: {len(valid_entries)}")
    
    # Write cleaned data back
    cleaned_filename = "evaluation_data.jsonl"
    with open(cleaned_filename, 'w') as f:
        for entry in valid_entries:
            f.write(json.dumps(entry) + '\n')
    
    print(f"Cleaned data saved to: {cleaned_filename}")
    return cleaned_filename

# Clean the evaluation data
cleaned_file = filter_valid_agent_data()

Line 1: Valid agent interaction found
Line 2: Valid agent interaction found
Line 3: Valid agent interaction found
Line 4: Valid agent interaction found
Line 5: Valid agent interaction found
Line 6: Valid agent interaction found
Line 7: Valid agent interaction found
Line 8: Valid agent interaction found
Line 9: Valid agent interaction found
Line 10: Valid agent interaction found

Total valid entries for agent evaluation: 10
Cleaned data saved to: evaluation_data.jsonl


### Upload a local JSONL file. Skip this step if you already have a dataset registered.


In [93]:
# Upload a local JSONL file. Skip this step if you already have a dataset registered.
dataset_name    = os.environ.get("DATASET_NAME",    "dataset-test")
dataset_version = os.environ.get("DATASET_VERSION", "1.7")
try:
    data_id = project_client.datasets.upload_file(
        name=dataset_name,
        version=dataset_version,
        file_path="./evaluation_data.jsonl",
    ).id
except Exception as e:
    # If the error is due to version already existing, fetch the dataset id
    print(f"Upload failed: {e}. Trying to fetch existing dataset version...")
    data_id = project_client.datasets.get(name=dataset_name, version=dataset_version).id
    print(f"Dataset '{dataset_name}' version '{dataset_version}' is ready. Dataset ID: {data_id}")

In [94]:
from azure.ai.evaluation import evaluate, AzureAIProject
from azure.ai.projects.models import (
    EvaluatorConfiguration,
    EvaluatorIds,
    Evaluation,
    InputDataset
)

# Built-in evaluator configurations:
evaluators = {
    "relevance": EvaluatorConfiguration(
        id=EvaluatorIds.RELEVANCE.value,
        init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
        data_mapping={
            "query": "${data.query}",
            "response": "${data.response}",
        },
    ),
    "violence": EvaluatorConfiguration(
        id=EvaluatorIds.VIOLENCE.value,
        init_params={"azure_ai_project": os.environ["AZURE_AI_FOUNDRY_ENDPOINT"]},
    ),
    "bleu_score": EvaluatorConfiguration(
        id=EvaluatorIds.BLEU_SCORE.value,
    ),
    # Agent-specific evaluators
    "intent_resolution": EvaluatorConfiguration(
        id=EvaluatorIds.INTENT_RESOLUTION.value,
        init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
        data_mapping={
            "query": "${data.query}",
            "response": "${data.response}",
        },
    ),
    "task_adherence": EvaluatorConfiguration(
        id=EvaluatorIds.TASK_ADHERENCE.value,
        init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
        data_mapping={
            "query": "${data.query}",
            "response": "${data.response}",
        },
    ),
    "tool_call_accuracy": EvaluatorConfiguration(
        id=EvaluatorIds.TOOL_CALL_ACCURACY.value,
        init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
        data_mapping={
            "query": "${data.query}",
            "response": "${data.response}",
            "tool_definitions": "${data.tool_definitions}",
        },
    ),
}

# Create an evaluation with the dataset and evaluators specified.
evaluation = Evaluation(
    display_name="Seattle Weather Agent evaluation",
    description="Evaluation of dataset",
    data=InputDataset(id=data_id),
    evaluators=evaluators,
)

# Run the evaluation.
evaluation_response = project_client.evaluations.create(
    evaluation,
    headers={
        "model-endpoint": os.environ["AZURE_OPENAI_ENDPOINT"],
        "api-key": os.getenv("AZURE_OPENAI_API_KEY"),
    },
)

print("Created evaluation:", evaluation_response.name)
print("Status:", evaluation_response.status)

Created evaluation: ef4f67b3-aad3-42d9-a515-77f61b207f20
Status: NotStarted


In [99]:
import json
import time

def print_eval_metrics(metrics):
    """Clean, formatted display of evaluation metrics"""
    if not metrics:
        print("No evaluation metrics available yet.")
        return

    # Handle case where metrics might be a JSON string
    if isinstance(metrics, str):
        try:
            metrics = json.loads(metrics)
        except json.JSONDecodeError:
            print("Raw metrics data:")
            print(metrics)
            return

    print("🔍 EVALUATION RESULTS")
    print("=" * 50)

    if isinstance(metrics, dict):
        for metric, value in metrics.items():
            if isinstance(value, dict):
                print(f"\n📊 {metric.replace('_', ' ').upper()}")
                for k, v in value.items():
                    if isinstance(v, (int, float)):
                        print(f"   {k}: {v:.3f}")
                    else:
                        print(f"   {k}: {v}")
            else:
                if isinstance(value, (int, float)):
                    print(f"📈 {metric.replace('_', ' ').upper()}: {value:.3f}")
                else:
                    print(f"📈 {metric.replace('_', ' ').upper()}: {value}")
    else:
        print("Raw metrics data:")
        print(metrics)

    print("=" * 50)

# Get and display metrics
eval_run = project_client.evaluations.get(name=evaluation_response.name)

# Wait for evaluation to complete
while eval_run.status not in ("Completed", "Failed", "Canceled"):
    print(f"Waiting for evaluation to complete. Current status: {eval_run.status}")
    time.sleep(10)
    eval_run = project_client.evaluations.get(name=evaluation_response.name)

print(f"Final evaluation status: {eval_run.status}")

if eval_run.status == "Failed":
    print("⚠️ Evaluation failed.")
    if hasattr(eval_run, "outputs"):
        print("Outputs:", eval_run.outputs)
    if hasattr(eval_run, "properties"):
        print("Properties:", eval_run.properties)
    if hasattr(eval_run, "systemData"):
        print("System Data:", eval_run.systemData)

metrics = eval_run.get('outputs', {}).get('evaluationMetrics', {})

print(f"Type of metrics: {type(metrics)}")
print_eval_metrics(metrics)

Final evaluation status: Completed
Type of metrics: <class 'str'>
🔍 EVALUATION RESULTS
📈 RELEVANCE.RELEVANCE: 3.700
📈 RELEVANCE.GPT RELEVANCE: 3.700
📈 RELEVANCE.RELEVANCE THRESHOLD: 3.000
📈 VIOLENCE.VIOLENCE THRESHOLD: 3.000
📈 INTENT RESOLUTION.INTENT RESOLUTION: 3.429
📈 INTENT RESOLUTION.INTENT RESOLUTION THRESHOLD: 3.000
📈 TASK ADHERENCE.TASK ADHERENCE: 4.100
📈 TASK ADHERENCE.TASK ADHERENCE THRESHOLD: 3.000
📈 TOOL CALL ACCURACY.TOOL CALL ACCURACY: 0.833
📈 TOOL CALL ACCURACY.TOOL CALL ACCURACY THRESHOLD: 0.800
📈 VIOLENCE.VIOLENCE DEFECT RATE: 0.000
📈 RELEVANCE.BINARY AGGREGATE: 0.900
📈 VIOLENCE.BINARY AGGREGATE: 1.000
📈 INTENT RESOLUTION.BINARY AGGREGATE: 0.500
📈 TASK ADHERENCE.BINARY AGGREGATE: 0.900
📈 TOOL CALL ACCURACY.BINARY AGGREGATE: 0.300


## Organized Multi-Evaluator Pattern

Instead of running all evaluators together, we can organize them into focused evaluation suites for better analysis and comparison. This approach allows for:

1. **Targeted Analysis**: Group related metrics for specific aspects
2. **Performance Comparison**: Run evaluations in parallel for faster results  
3. **Clear Reporting**: Separate results by evaluation focus area
4. **Iterative Testing**: Test specific areas during development

## Inspect results on Azure AI Foundry

Go to AI Foundry URL for rich Azure AI Foundry data visualization to inspect the evaluation scores and reasoning to quickly identify bugs and issues of your agent to fix and improve.

In [100]:
import asyncio
import concurrent.futures
from typing import Dict, List, Any
import pandas as pd

class OrganizedEvaluationSuite:
    """
    Organized evaluation suite that groups evaluators by focus area
    and provides comprehensive comparison and reporting capabilities.
    """
    
    def __init__(self, project_client, dataset_path="./evaluation_data.jsonl"):
        self.project_client = project_client
        self.dataset_path = dataset_path
        self.evaluation_results = {}
        
    def get_safety_evaluators(self) -> Dict[str, EvaluatorConfiguration]:
        """Safety and content moderation evaluators"""
        return {
            "violence": EvaluatorConfiguration(
                id=EvaluatorIds.VIOLENCE.value,
                init_params={"azure_ai_project": os.environ["AZURE_AI_FOUNDRY_ENDPOINT"]},
            ),
            "hate_unfairness": EvaluatorConfiguration(
                id=EvaluatorIds.HATE_UNFAIRNESS.value,
                init_params={"azure_ai_project": os.environ["AZURE_AI_FOUNDRY_ENDPOINT"]},
            ),
            "self_harm": EvaluatorConfiguration(
                id=EvaluatorIds.SELF_HARM.value,
                init_params={"azure_ai_project": os.environ["AZURE_AI_FOUNDRY_ENDPOINT"]},
            ),
        }
    
    def get_quality_evaluators(self) -> Dict[str, EvaluatorConfiguration]:
        """General response quality evaluators"""
        return {
            "relevance": EvaluatorConfiguration(
                id=EvaluatorIds.RELEVANCE.value,
                init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
                data_mapping={
                    "query": "${data.query}",
                    "response": "${data.response}",
                },
            ),
            "coherence": EvaluatorConfiguration(
                id=EvaluatorIds.COHERENCE.value,
                init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
                data_mapping={
                    "query": "${data.query}",
                    "response": "${data.response}",
                    "tools": "${data.tool_definitions}",
                },
            ),
            "fluency": EvaluatorConfiguration(
                id=EvaluatorIds.FLUENCY.value,
                init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
                data_mapping={
                    "query": "${data.query}",
                    "response": "${data.response}",
                },
            ),
        }
    
    def get_agent_behavior_evaluators(self) -> Dict[str, EvaluatorConfiguration]:
        """Agent-specific behavioral evaluators"""
        return {
            "intent_resolution": EvaluatorConfiguration(
                id=EvaluatorIds.INTENT_RESOLUTION.value,
                init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
                data_mapping={
                    "query": "${data.query}",
                    "response": "${data.response}",
                },
            ),
            "task_adherence": EvaluatorConfiguration(
                id=EvaluatorIds.TASK_ADHERENCE.value,
                init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
                data_mapping={
                    "query": "${data.query}",
                    "response": "${data.response}",
                },
            ),
        }
    
    def get_technical_evaluators(self) -> Dict[str, EvaluatorConfiguration]:
        """Technical capability evaluators"""
        return {
            "tool_call_accuracy": EvaluatorConfiguration(
                id=EvaluatorIds.TOOL_CALL_ACCURACY.value,
                init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
                data_mapping={
                    "conversation": "${data.query}",
                    "response": "${data.response}",
                    "tools": "${data.tool_definitions}",
                },
            ),
        }
    
    def run_evaluation_suite(self, suite_name: str, evaluators: Dict[str, EvaluatorConfiguration]) -> str:
        """Run a specific evaluation suite"""
        print(f"🚀 Running {suite_name} Evaluation Suite...")
        
        # Upload dataset
        dataset_name = f"agent-eval-{suite_name.lower().replace(' ', '-')}"
        dataset_version = "1.1"

        try:
            data_id = self.project_client.datasets.upload_file(
                name=dataset_name,
                version=dataset_version,
                file_path=self.dataset_path,
            ).id
        except Exception as e:
            # If the error is due to version already existing, fetch the dataset id
            print(f"Upload failed: {e}. Trying to fetch existing dataset version...")
            data_id = self.project_client.datasets.get(name=dataset_name, version=dataset_version).id
            print(f"Dataset '{dataset_name}' version '{dataset_version}' is ready. Dataset ID: {data_id}")
        # Create evaluation
        evaluation = Evaluation(
            display_name=f"{suite_name} - Agent Quality Assessment",
            description=f"Focused evaluation of agent {suite_name.lower()} capabilities",
            data=InputDataset(id=data_id),
            evaluators=evaluators,
        )
        
        # Run evaluation
        evaluation_response = self.project_client.evaluations.create(
            evaluation,
            headers={
                "model-endpoint": os.environ["AZURE_OPENAI_ENDPOINT"],
                "api-key": os.getenv("AZURE_OPENAI_API_KEY"),
            },
        )
        
        print(f"✅ {suite_name} evaluation started: {evaluation_response.name}")
        print(f"   Status: {evaluation_response.status}")
        
        return evaluation_response.name
    
    def run_all_evaluation_suites(self) -> Dict[str, str]:
        """Run all evaluation suites in parallel"""
        print("🔄 Starting Comprehensive Agent Evaluation...\n")
        
        suites = {
            "Safety": self.get_safety_evaluators(),
            "Quality": self.get_quality_evaluators(), 
            "Agent Behavior": self.get_agent_behavior_evaluators(),
            "Technical": self.get_technical_evaluators(),
        }
        
        evaluation_ids = {}
        
        # Run all suites
        for suite_name, evaluators in suites.items():
            try:
                eval_id = self.run_evaluation_suite(suite_name, evaluators)
                evaluation_ids[suite_name] = eval_id
                print()  # Add spacing
            except Exception as e:
                print(f"❌ Failed to start {suite_name} evaluation: {str(e)}")
                print()
        
        return evaluation_ids
    
    def get_evaluation_comparison_report(self, evaluation_ids: Dict[str, str]) -> pd.DataFrame:
        """Generate a comparison report across all evaluation suites"""
        print("📊 Generating Evaluation Comparison Report...\n")
        
        results = []
        
        for suite_name, eval_id in evaluation_ids.items():
            try:
                eval_run = self.project_client.evaluations.get(name=eval_id)
                metrics = eval_run.get('outputs', {}).get('evaluationMetrics', {})
                
                # Extract key metrics for comparison
                suite_results = {
                    'Suite': suite_name,
                    'Evaluation_ID': eval_id,
                    'Status': eval_run.get('status', 'Unknown'),
                }
                
                # Add metrics if available
                if isinstance(metrics, dict):
                    for metric_name, metric_value in metrics.items():
                        if isinstance(metric_value, (int, float)):
                            suite_results[f'{metric_name}_score'] = metric_value
                        elif isinstance(metric_value, dict) and 'mean' in metric_value:
                            suite_results[f'{metric_name}_mean'] = metric_value['mean']
                
                results.append(suite_results)
                
            except Exception as e:
                print(f"⚠️  Could not retrieve results for {suite_name}: {str(e)}")
                results.append({
                    'Suite': suite_name,
                    'Evaluation_ID': eval_id,
                    'Status': 'Error retrieving results',
                })
        
        df = pd.DataFrame(results)
        
        print("📋 Evaluation Suite Comparison:")
        print("=" * 60)
        display(df)
        
        return df
    
    def print_evaluation_summary(self, comparison_df: pd.DataFrame):
        """Print a comprehensive evaluation summary"""
        print("\n🎯 AGENT EVALUATION SUMMARY")
        print("=" * 60)
        
        completed_suites = comparison_df[comparison_df['Status'] == 'Completed']
        
        if len(completed_suites) > 0:
            print(f"✅ Completed Evaluations: {len(completed_suites)}/{len(comparison_df)}")
            
            # Show key metrics if available
            metric_columns = [col for col in comparison_df.columns if col.endswith('_score') or col.endswith('_mean')]
            
            if metric_columns:
                print(f"\n📈 Key Performance Indicators:")
                for _, row in completed_suites.iterrows():
                    print(f"\n   {row['Suite']}:")
                    for metric in metric_columns:
                        if pd.notna(row.get(metric)):
                            metric_name = metric.replace('_score', '').replace('_mean', '').title()
                            print(f"     • {metric_name}: {row[metric]:.3f}")
                
                # Overall assessment
                print(f"\n🔍 Overall Assessment:")
                print(f"   • Safety: Check all safety metrics are within acceptable thresholds")
                print(f"   • Quality: Ensure relevance, coherence, and fluency scores are high")
                print(f"   • Agent Behavior: Verify intent resolution and task adherence")
                print(f"   • Technical: Confirm tool usage accuracy")
            
        else:
            print("⏳ No completed evaluations yet. Check back later for results.")
        
        print("\n" + "=" * 60)

# Initialize the organized evaluation suite
eval_suite = OrganizedEvaluationSuite(project_client)

In [102]:
# Run comprehensive evaluation across all suites
evaluation_ids = eval_suite.run_all_evaluation_suites()

🔄 Starting Comprehensive Agent Evaluation...

🚀 Running Safety Evaluation Suite...
Upload failed: (UserError) Asset azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-safety/versions/1.1 already exists, cannot create new temporary data references for it
Code: UserError
Message: Asset azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-safety/versions/1.1 already exists, cannot create new temporary data references for it. Trying to fetch existing dataset version...
Dataset 'agent-eval-safety' version '1.1' is ready. Dataset ID: azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-safety/versions/1.1
✅ Safety evaluation started: 5a05fac6-860f-4dfe-9d4c-6acd09370636
   Status: NotStarted

🚀 Running Quality Evaluation Suite...
Upload failed: (UserError) Asset azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-qu

In [103]:
# Generate comparison report and summary (run this after evaluations complete)
comparison_df = eval_suite.get_evaluation_comparison_report(evaluation_ids)
eval_suite.print_evaluation_summary(comparison_df)

📊 Generating Evaluation Comparison Report...

📋 Evaluation Suite Comparison:


,Suite,Evaluation_ID,Status
0,Safety,5a05fac6-860f-4dfe-9d4c-6acd09370636,Completed
1,Quality,2525f582-a2dd-4ff4-93f1-ba38ecaa6cc9,Completed
2,Agent Behavior,f2f40444-b714-44e8-af84-243cd1fce451,Completed
3,Technical,708bb6c4-75aa-423a-9108-4f4d8f48c389,Completed



🎯 AGENT EVALUATION SUMMARY
✅ Completed Evaluations: 4/4



### Advanced Evaluation Patterns

#### 1. **Targeted Suite Evaluation**
Run specific evaluation suites for focused testing during development:

In [104]:
from azure.core.exceptions import ResourceExistsError

# Example: Run only safety evaluation during development
safety_evaluators = eval_suite.get_safety_evaluators()
try:
	safety_eval_id = eval_suite.run_evaluation_suite("Safety", safety_evaluators)
except ResourceExistsError:
	print("Safety dataset already exists. Using the existing version.")
	dataset_name = "agent-eval-safety"
	dataset_version = "1.1"
	safety_eval_id = project_client.datasets.get(name=dataset_name, version=dataset_version).id

# Example: Test agent behavior specifically
agent_evaluators = eval_suite.get_agent_behavior_evaluators()
try:
	agent_eval_id = eval_suite.run_evaluation_suite("Agent Behavior", agent_evaluators)
except ResourceExistsError:
	print("Agent Behavior dataset already exists. Using the existing version.")
	dataset_name = "agent-eval-agent-behavior"
	dataset_version = "1.1"
	agent_eval_id = project_client.datasets.get(name=dataset_name, version=dataset_version).id

🚀 Running Safety Evaluation Suite...
Upload failed: (UserError) Asset azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-safety/versions/1.1 already exists, cannot create new temporary data references for it
Code: UserError
Message: Asset azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-safety/versions/1.1 already exists, cannot create new temporary data references for it. Trying to fetch existing dataset version...
Dataset 'agent-eval-safety' version '1.1' is ready. Dataset ID: azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-safety/versions/1.1
✅ Safety evaluation started: f363d169-9c07-45b3-bccc-35edc485dbee
   Status: NotStarted
🚀 Running Agent Behavior Evaluation Suite...
Upload failed: (UserError) Asset azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-agent-behavior/versions/1.1 already exists

#### 2. **Custom Evaluation Suite Creation**
Create your own evaluation suites for specific use cases:

In [ ]:
# Custom evaluation suite for production readiness
def get_production_readiness_evaluators() -> Dict[str, EvaluatorConfiguration]:
    """Combined evaluators for production readiness assessment"""
    return {
        # Safety first
        "violence": EvaluatorConfiguration(
            id=EvaluatorIds.VIOLENCE.value,
            init_params={"azure_ai_project": os.environ["AZURE_AI_FOUNDRY_ENDPOINT"]},
        ),
        "hate_unfairness": EvaluatorConfiguration(
            id=EvaluatorIds.HATE_UNFAIRNESS.value,
            init_params={"azure_ai_project": os.environ["AZURE_AI_FOUNDRY_ENDPOINT"]},
        ),
        # Quality assurance
        "relevance": EvaluatorConfiguration(
            id=EvaluatorIds.RELEVANCE.value,
            init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
            data_mapping={
                "query": "${data.query}",
                "response": "${data.response}",
            },
        ),
        # Agent behavior
        "intent_resolution": EvaluatorConfiguration(
            id=EvaluatorIds.INTENT_RESOLUTION.value,
            init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
            data_mapping={
                "query": "${data.query}",
                "response": "${data.response}",
            },
        ),
        "task_adherence": EvaluatorConfiguration(
            id=EvaluatorIds.TASK_ADHERENCE.value,
            init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
            data_mapping={
                "query": "${data.query}",
                "response": "${data.response}",
            },
        ),
        # Technical capability
        "tool_call_accuracy": EvaluatorConfiguration(
            id=EvaluatorIds.TOOL_CALL_ACCURACY.value,
            init_params={"deployment_name": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]},
            data_mapping={
                "query": "${data.query}",
                "response": "${data.response}",
                "tool_definitions": "${data.tool_definitions}",
            },
        ),
    }

# Run production readiness evaluation
production_evaluators = get_production_readiness_evaluators()
production_eval_id = eval_suite.run_evaluation_suite("Production Readiness", production_evaluators)

🚀 Running Production Readiness2 Evaluation Suite...
Upload failed: (UserError) Asset azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-production-readiness2/versions/1.1 already exists, cannot create new temporary data references for it
Code: UserError
Message: Asset azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-production-readiness2/versions/1.1 already exists, cannot create new temporary data references for it. Trying to fetch existing dataset version...
Dataset 'agent-eval-production-readiness2' version '1.1' is ready. Dataset ID: azureai://accounts/aifoundry825233136833-resource/projects/aifoundry825233136833/data/agent-eval-production-readiness2/versions/1.1
✅ Production Readiness2 evaluation started: ce615cdf-7850-49e3-bb2a-dc953eebe9a5
   Status: NotStarted


#### 3. **Evaluation Results Analysis**
Advanced analysis and comparison utilities:

In [110]:
import matplotlib.pyplot as plt
import seaborn as sns

def create_evaluation_dashboard(comparison_df: pd.DataFrame):
    """Create a visual dashboard for evaluation results"""
    
    # Filter for completed evaluations with numeric metrics
    completed = comparison_df[comparison_df['Status'] == 'Completed']
    
    if len(completed) == 0:
        print("⏳ No completed evaluations to visualize yet.")
        return
    
    # Extract metric columns
    metric_cols = [col for col in completed.columns if col.endswith('_score') or col.endswith('_mean')]
    
    if not metric_cols:
        print("📊 No numeric metrics available for visualization yet.")
        return
    
    # Create dashboard
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Agent Evaluation Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Suite completion status
    status_counts = comparison_df['Status'].value_counts()
    axes[0,0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0,0].set_title('Evaluation Suite Status')
    
    # 2. Metric comparison by suite (if metrics available)
    if len(metric_cols) > 0:
        completed_metrics = completed[['Suite'] + metric_cols].set_index('Suite')
        completed_metrics.plot(kind='bar', ax=axes[0,1], rot=45)
        axes[0,1].set_title('Metrics by Evaluation Suite')
        axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        axes[0,1].text(0.5, 0.5, 'Metrics not available yet', ha='center', va='center')
        axes[0,1].set_title('Metrics by Evaluation Suite')
    
    # 3. Overall performance radar (if enough metrics)
    if len(completed) > 0 and len(metric_cols) >= 3:
        # Create a simple score overview
        suite_names = completed['Suite'].tolist()
        avg_scores = []
        
        for _, row in completed.iterrows():
            suite_scores = [row[col] for col in metric_cols if pd.notna(row[col])]
            if suite_scores:
                avg_scores.append(sum(suite_scores) / len(suite_scores))
            else:
                avg_scores.append(0)
        
        axes[1,0].barh(suite_names, avg_scores)
        axes[1,0].set_title('Average Performance by Suite')
        axes[1,0].set_xlabel('Average Score')
        
        # 4. Detailed metrics heatmap
        if len(completed) > 1:
            heatmap_data = completed[['Suite'] + metric_cols].set_index('Suite')
            sns.heatmap(heatmap_data, annot=True, fmt='.3f', ax=axes[1,1], cmap='RdYlGn')
            axes[1,1].set_title('Detailed Metrics Heatmap')
        else:
            axes[1,1].text(0.5, 0.5, 'Need multiple suites\nfor comparison', ha='center', va='center')
            axes[1,1].set_title('Detailed Metrics Heatmap')
    else:
        axes[1,0].text(0.5, 0.5, 'Insufficient data\nfor radar chart', ha='center', va='center')
        axes[1,0].set_title('Average Performance by Suite')
        
        axes[1,1].text(0.5, 0.5, 'Insufficient data\nfor heatmap', ha='center', va='center')
        axes[1,1].set_title('Detailed Metrics Heatmap')
    
    plt.tight_layout()
    plt.show()

def generate_evaluation_insights(comparison_df: pd.DataFrame):
    """Generate insights and recommendations from evaluation results"""
    print("🧠 EVALUATION INSIGHTS & RECOMMENDATIONS")
    print("=" * 60)
    
    completed = comparison_df[comparison_df['Status'] == 'Completed']
    
    if len(completed) == 0:
        print("⏳ Complete evaluations first to generate insights.")
        return
    
    # Analyze by suite
    for _, row in completed.iterrows():
        suite_name = row['Suite']
        print(f"\n📋 {suite_name} Suite Analysis:")
        
        if suite_name == 'Safety':
            print("   🛡️  Safety is critical for production deployment")
            print("   📊 All safety scores should be close to 0 (low risk)")
            print("   ⚠️  Any non-zero safety scores need immediate attention")
        
        elif suite_name == 'Quality':
            print("   🎯 Quality metrics indicate user satisfaction potential")
            print("   📈 Target: Relevance > 0.8, Coherence > 0.7, Fluency > 0.8") 
            print("   💡 Low scores suggest need for prompt engineering or model tuning")
        
        elif suite_name == 'Agent Behavior':
            print("   🤖 Measures how well agent follows instructions and resolves intents")
            print("   🎯 Target: Intent Resolution > 4.0, Task Adherence > 4.0 (out of 5)")
            print("   🔧 Low scores indicate need for better system prompts or training")
        
        elif suite_name == 'Technical':
            print("   ⚙️  Evaluates technical execution capability")
            print("   🎯 Target: Tool Call Accuracy > 0.9")
            print("   🛠️  Low scores suggest tool definition or reasoning issues")
        
        # Show specific metrics if available
        metric_cols = [col for col in comparison_df.columns if col.endswith('_score') or col.endswith('_mean')]
        for col in metric_cols:
            if pd.notna(row[col]):
                metric_name = col.replace('_score', '').replace('_mean', '')
                score = row[col]
                print(f"      • {metric_name}: {score:.3f}")
    
    print(f"\n💡 Next Steps:")
    print(f"   1. Address any safety concerns immediately")
    print(f"   2. Iterate on areas with lowest quality scores")
    print(f"   3. Run focused evaluations during development")
    print(f"   4. Set up continuous evaluation in your CI/CD pipeline")
    print("=" * 60)

In [112]:
# Create visual dashboard and insights (run after evaluations complete)
create_evaluation_dashboard(comparison_df)
generate_evaluation_insights(comparison_df)

📊 No numeric metrics available for visualization yet.
🧠 EVALUATION INSIGHTS & RECOMMENDATIONS

📋 Safety Suite Analysis:
   🛡️  Safety is critical for production deployment
   📊 All safety scores should be close to 0 (low risk)
   ⚠️  Any non-zero safety scores need immediate attention

📋 Quality Suite Analysis:
   🎯 Quality metrics indicate user satisfaction potential
   📈 Target: Relevance > 0.8, Coherence > 0.7, Fluency > 0.8
   💡 Low scores suggest need for prompt engineering or model tuning

📋 Agent Behavior Suite Analysis:
   🤖 Measures how well agent follows instructions and resolves intents
   🎯 Target: Intent Resolution > 4.0, Task Adherence > 4.0 (out of 5)
   🔧 Low scores indicate need for better system prompts or training

📋 Technical Suite Analysis:
   ⚙️  Evaluates technical execution capability
   🎯 Target: Tool Call Accuracy > 0.9
   🛠️  Low scores suggest tool definition or reasoning issues

💡 Next Steps:
   1. Address any safety concerns immediately
   2. Iterate on ar